In [41]:
import numpy as np
import math
import sys

In [42]:

def kernel(x,y):
    x = np.array(x)
    y = np.array(y)
    return np.exp((-(x-y)**2)/2)

In [43]:
def computeMMD(x, y, N,i,j):
    # Convert x and y to numpy arrays
    
    
    # Compute kernel matrices using broadcasting
    Kxx = kernel(x[:, None], x[None, :])  # Kernel matrix for x with itself
    Kyy = kernel(y[:, None], y[None, :])  # Kernel matrix for y with itself
    Kxy = kernel(x[:, None], y[None, :])  # Kernel matrix for x with y

    # Calculate the MMD using vectorized operations
    MMD_squared = (np.sum(Kxx) + np.sum(Kyy) - 2 * np.sum(Kxy)) / (N**2)
    print("i,j,MMD_squared is :",i,j, MMD_squared)
    
    return np.sqrt(MMD_squared)


In [44]:
#lst = [[0,10,6,5,9],[10,0,8,7,2],[6,8,0,1,4],[5,7,1,0,3],[9,2,4,3,0]]
import random
M = 25      # total number of sequences
Mk = 5      # number of sequences per cluster
K = 5       # number of clusters
Nvec= [1500]     # number of elements in 1 sequence
clusters = [[i] for i in range(M)]
# means = [0.4,0.55,0.7,0.85,1,1.15,1.3,1.45,1.6, 1.85,2,2.15]
means = [i for i in range(K) for j in range(Mk)]
# correct = [[0,1,2,3,4,5,6,7,8],[9,10,11]]
correct = [[(Mk*i)+j for j in range(Mk)]for i in range(K)]
print(means)
print(correct)
# with open('slink-fss-mmdnew100.txt', 'a') as f:
#     print(correct,means, file = f)

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4]
[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19], [20, 21, 22, 23, 24]]


In [45]:
Pe = np.zeros(len(Nvec))  # Initializing Pe array with zeros
lnPe = np.zeros(len(Nvec))  # Initializing lnPe array with zeros

for ii in range(len(Nvec)):  # Looping through each element in Nvec
    
    N = Nvec[ii]  # Setting N to the current element in Nvec
    total = 0  # Initializing total to 0
    error = 0  # Initializing error to 0
    while error < 10:  # Loop until error is less than 100

        clusters = [[i] for i in range(M)]  # Initializing clusters
        total += 1  # Incrementing total by 1
        lst = [[] for i in range(M)]  # Initializing lst
        # print("the length of the list is",len(lst))
        a = [np.random.normal(means[i], 1, (1, N)) for i in range(M)]  # Generating random samples
        y = np.asarray(a)  # Converting a to numpy array
        y = np.vstack(y)  # Stacking arrays in sequence vertically
        lst = [[computeMMD(y[i, :], y[j, :], N,i,j) for j in range(i + 1)] for i in range(M)]  # Computing MMD

        for i in range(M):  # Looping through each element in M
            for j in range(M):  # Looping through each element in M
                if i < j:  # If i is less than j
                    lst[i].append(lst[j][i])  # Append lst[j][i] to lst[i]
        
        while len(clusters) > K:  # Loop until the length of clusters is greater than K

####################### FIND MAXIMUM DISTANCE FOR COMPLETE LINKAGE ####
            max_dist = -sys.float_info.max  # Initializing max_dist to the smallest possible float value
            for i in range(len(lst)):  # Looping through each element in lst
                for j in range(i + 1):  # Looping through each element in lst up to i
                    if i != j:  
                        if lst[i][j] > max_dist:  # If lst[i][j] is greater than max_dist
                            max_dist = lst[i][j]  # Set max_dist to lst[i][j]
                            a, b = i, j  # Set a and b to i and j respectively
            
            clusters[b].extend(clusters[a])  # Extend clusters[b] with clusters[a]
            del clusters[a]  # Delete clusters[a]
            print("the final cluster is :",clusters)



            
            #### MERGE ROWS ####
            for i in range(len(lst[b])):  # Looping through each element in lst[b]
                if max(lst[a][i], lst[b][i]) != 0:  # If the maximum of lst[a][i] and lst[b][i] is not 0
                    lst[b][i] = max(lst[a][i], lst[b][i])  # Set lst[b][i] to the maximum of lst[a][i] and lst[b][i]
            del lst[a]  




            ############################ UPDATE ROWS ##########################
            for i in range(len(lst)):  # Looping through each element in lst
                if i != b:
                    lst[i][b] = max(lst[i][a], lst[i][b]) 
                del lst[i][a]  # Delete lst[i][a]

        clusters.sort()  # Sort clusters
        C_sort = [] 
        for x in clusters:  # Looping through each element in clusters
            x.sort()  # Sort x
            C_sort.append(x)  

        if correct != C_sort: 
            error = error + 1 

        print(error, total) 

    Pe[ii] = error / total 
    lnPe[ii] = math.log(Pe[ii])  
    print(total, Pe[ii], lnPe[ii])  

i,j,MMD_squared is : 0 0 0.0
i,j,MMD_squared is : 1 0 0.0002929605616326961
i,j,MMD_squared is : 1 1 0.0
i,j,MMD_squared is : 2 0 0.0004184270666340987
i,j,MMD_squared is : 2 1 0.0004525713971491075
i,j,MMD_squared is : 2 2 0.0
i,j,MMD_squared is : 3 0 0.00024520321608417566
i,j,MMD_squared is : 3 1 0.00040378442561688524
i,j,MMD_squared is : 3 2 0.0001388315535326385
i,j,MMD_squared is : 3 3 0.0
i,j,MMD_squared is : 4 0 2.9675113815193374e-05
i,j,MMD_squared is : 4 1 0.0002819517787554198
i,j,MMD_squared is : 4 2 0.000299718493975699
i,j,MMD_squared is : 4 3 0.0001767446017091473
i,j,MMD_squared is : 4 4 0.0
i,j,MMD_squared is : 5 0 0.1543291247523516
i,j,MMD_squared is : 5 1 0.16040465765592268
i,j,MMD_squared is : 5 2 0.16396415216546134
i,j,MMD_squared is : 5 3 0.1648492968277921
i,j,MMD_squared is : 5 4 0.15516758406710934
i,j,MMD_squared is : 5 5 0.0
i,j,MMD_squared is : 6 0 0.15474144157347455
i,j,MMD_squared is : 6 1 0.16086653651048574
i,j,MMD_squared is : 6 2 0.16431741670206